In [25]:
import pandas as pd
import win32com.client as win32
from pandas_datareader import data
import matplotlib.pyplot as plt
import numpy as np
import urllib.request
from datetime import date
from datetime import timedelta
from bs4 import BeautifulSoup

In [26]:
tickers_page_address = 'https://en.wikipedia.org/wiki/S%26P_100'

In [27]:
tickers_page_html = urllib.request.urlopen(tickers_page_address)

In [28]:
soup = BeautifulSoup(tickers_page_html,'html.parser')

In [29]:
all_ticker_text = soup.find_all('td')

In [30]:
tickers = []
names = []
for x in range(1, 100):
    tickers.append((all_ticker_text[x*2+13].get_text())[:-1])
    names.append((all_ticker_text[x*2+12].get_text())[:-1])

In [31]:
print(tickers)


['AAPL', 'ABBV', 'ABT', 'ACN', 'AGN', 'AIG', 'ALL', 'AMGN', 'AMZN', 'AXP', 'BA', 'BAC', 'BIIB', 'BK', 'BKNG', 'BLK', 'BMY', 'BRK.B', 'C', 'CAT', 'CELG', 'CHTR', 'CL', 'CMCSA', 'COF', 'COP', 'COST', 'CSCO', 'CVS', 'CVX', 'DHR', 'DIS', 'DUK', 'DOW', 'DWDP', 'EMR', 'EXC', 'F', 'FB', 'FDX', 'FOX', 'FOXA', 'GD', 'GE', 'GILD', 'GM', 'GOOG', 'GOOGL', 'GS', 'HD', 'HON', 'IBM', 'INTC', 'JNJ', 'JPM', 'KHC', 'KMI', 'KO', 'LLY', 'LMT', 'LOW', 'MA', 'MCD', 'MDLZ', 'MDT', 'MET', 'MMM', 'MO', 'MRK', 'MS', 'MSFT', 'NEE', 'NFLX', 'NKE', 'NVDA', 'ORCL', 'OXY', 'PEP', 'PFE', 'PG', 'PM', 'PYPL', 'QCOM', 'RTN', 'SBUX', 'SLB', 'SO', 'SPG', 'T', 'TGT', 'TXN', 'UNH', 'UNP', 'UPS', 'USB', 'UTX', 'V', 'VZ', 'WBA']


In [32]:
today = str(date.today())
start_date = str(date.today()-timedelta(days=5))
print(today)
print(start_date)

2019-04-08
2019-04-03


In [33]:
price_data = []
valid_tickers = []
valid_names = []
for i in range(0, 99):
    try: 
        print(tickers[i])
        price_data.append(data.DataReader(tickers[i],'iex',start_date,today).close)
        valid_tickers.append(tickers[i])
        valid_names.append(names[i])
    except: 
        pass

AAPL
ABBV
ABT
ACN
AGN
AIG
ALL
AMGN
AMZN
AXP
BA
BAC
BIIB
BK
BKNG
BLK
BMY
BRK.B
C
CAT
CELG
CHTR
CL
CMCSA
COF
COP
COST
CSCO
CVS
CVX
DHR
DIS
DUK
DOW
DWDP
EMR
EXC
F
FB
FDX
FOX
FOXA
GD
GE
GILD
GM
GOOG
GOOGL
GS
HD
HON
IBM
INTC
JNJ
JPM
KHC
KMI
KO
LLY
LMT
LOW
MA
MCD
MDLZ
MDT
MET
MMM
MO
MRK
MS
MSFT
NEE
NFLX
NKE
NVDA
ORCL
OXY
PEP
PFE
PG
PM
PYPL
QCOM
RTN
SBUX
SLB
SO
SPG
T
TGT
TXN
UNH
UNP
UPS
USB
UTX
V
VZ
WBA


In [34]:
df_price_data = pd.DataFrame(price_data)
movements = pd.DataFrame()
for i in range(0, len(valid_tickers)-1):
    percent_move = np.log(float(df_price_data.iloc[i,-1])/float(df_price_data.iloc[i,-2]))
    movements = movements.append({'ticker': valid_tickers[i], 'name': valid_names[i], 'change': "{0:.2f}%".format(percent_move*100), 'absChange': np.abs(percent_move)}, ignore_index=True)

In [35]:
movements_top10 = movements.sort_values(by=['absChange'], ascending=False)[:10]

In [36]:
movements_top10 = movements_top10.reset_index(drop=True)
movements_top10.index = movements_top10.index + 1

In [37]:
del movements_top10['absChange']

In [38]:
movements_top10 = movements_top10[['ticker','name','change']]
movements_top10.columns = ['Ticker','name',list(df_price_data)[-1]]

In [39]:
movements_top10

,Ticker,name,2019-04-05
1,DOW,Duke Energy,-4.22%
2,BIIB,Bank of America Corp,2.80%
3,FOX,FedEx,2.60%
4,FOXA,Fox Corporation B,2.47%
5,OXY,Oracle Corporation,2.38%
6,SLB,Starbucks Corp.,2.02%
7,CHTR,Celgene Corp,1.64%
8,AMGN,Allstate,1.59%
9,RTN,Qualcomm Inc.,1.50%
10,COP,Capital One Financial Corp.,1.46%


In [40]:
import win32com.client as win32
outlook = win32.Dispatch('outlook.application')
mail = outlook.CreateItem(0)
mail.To = 'james9995@hotmail.co.uk'
mail.Subject = 'S&P 100 Top 10 Movers '+list(df_price_data)[-1]
mail.Body = 'S&P 100 Top 10 Movers'
html1 = movements_top10.to_html()
mail.HTMLBody = html1
mail.Send()
